In [1]:
from IPython.display import Image
%matplotlib inline

In [2]:
#loads the Verne's MThe Mysterious Island
import numpy as np

## Reading and processing text
with open('1268-0.txt', 'r', encoding="utf8") as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')

text = text[start_indx:end_indx]
char_set = set(text)
print('Total Length:', len(text))
print('Unique Characters:', len(char_set))

Total Length: 1112350
Unique Characters: 80


In [3]:
#building a dictionary for mapping characters to integers
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)

print(text[:15], '     == Encoding ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == Reverse  ==> ', ''.join(char_array[text_encoded[15:21]]))

Text encoded shape:  (1112350,)
THE MYSTERIOUS       == Encoding ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == Reverse  ==>  ISLAND


In [4]:
#cuccok
#import torch
#from torch.utils.data import DataLoader
#import torch.nn as nn
#from torch.distributions.categorical import Categorical
print("Mindet beolvasta, el birok vele kezdeni cuccolni")

Mindet beolvasta, el birok vele kezdeni cuccolni


In [5]:
#print out the first five
for ex in text_encoded[:5]:
    print('{} -> {}'.format(ex, char_array[ex]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [6]:
#all the 80?
for ex in text_encoded[:80]:
    print('{} -> {}'.format(ex, char_array[ex]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M
48 -> Y
43 -> S
44 -> T
29 -> E
42 -> R
33 -> I
39 -> O
45 -> U
43 -> S
1 ->  
33 -> I
43 -> S
36 -> L
25 -> A
38 -> N
28 -> D
1 ->  
6 -> *
6 -> *
6 -> *
0 -> 

0 -> 

0 -> 

0 -> 

0 -> 

40 -> P
67 -> r
64 -> o
53 -> d
70 -> u
52 -> c
54 -> e
53 -> d
1 ->  
51 -> b
74 -> y
1 ->  
25 -> A
63 -> n
69 -> t
57 -> h
64 -> o
63 -> n
74 -> y
1 ->  
37 -> M
50 -> a
69 -> t
64 -> o
63 -> n
50 -> a
60 -> k
7 -> ,
1 ->  
50 -> a
63 -> n
53 -> d
1 ->  
44 -> T
67 -> r
54 -> e
71 -> v
64 -> o
67 -> r
1 ->  
27 -> C
50 -> a
67 -> r
61 -> l
68 -> s
64 -> o
63 -> n
0 -> 

0 -> 

0 -> 



In [7]:
seq_length = 40
chunk_size = seq_length + 1

text_chunks = [text_encoded[i:i+chunk_size] 
               for i in range(len(text_encoded)-chunk_size+1)] 

## inspection:
for seq in text_chunks[:1]:
    input_seq = seq[:seq_length]
    target = seq[seq_length] 
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  ->  74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  ->  'y'


In [8]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text_chunks):
        self.text_chunks = text_chunks

    def __len__(self):
        return len(self.text_chunks)
    
    def __getitem__(self, idx):
        text_chunk = self.text_chunks[idx]
        return text_chunk[:-1].long(), text_chunk[1:].long()
    
seq_dataset = TextDataset(torch.tensor(text_chunks))

/tmp/ipykernel_6866/2527503007.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /croot/pytorch_1681837265408/work/torch/csrc/utils/tensor_new.cpp:230.)
  seq_dataset = TextDataset(torch.tensor(text_chunks))


In [9]:
for i, (seq, target) in enumerate(seq_dataset):
    print(' Input (x):', repr(''.join(char_array[seq])))
    print('Target (y):', repr(''.join(char_array[target])))
    print()
    if i == 1:
        break
    

 Input (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
Target (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

 Input (x): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'
Target (y): 'E MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by '



In [13]:
#cuda available check
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [14]:
from torch.utils.data import DataLoader
 
batch_size = 64

torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [15]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) 
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x).unsqueeze(1)
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size)
        cell = torch.zeros(1, batch_size, self.rnn_hidden_size)
        return hidden.to(device), cell.to(device)
    
vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 512

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(80, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=80, bias=True)
)

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

num_epochs = 10000 

torch.manual_seed(1)

for epoch in range(num_epochs):
    hidden, cell = model.init_hidden(batch_size)
    seq_batch, target_batch = next(iter(seq_dl))
    seq_batch = seq_batch.to(device)
    target_batch = target_batch.to(device)
    optimizer.zero_grad()
    loss = 0
    for c in range(seq_length):
        pred, hidden, cell = model(seq_batch[:, c], hidden, cell) 
        loss += loss_fn(pred, target_batch[:, c])
    loss.backward()
    optimizer.step()
    loss = loss.item()/seq_length
    if epoch % 500 == 0:
        print(f'Epoch {epoch} loss: {loss:.4f}')
 

Epoch 0 loss: 4.3722
Epoch 500 loss: 1.3963
Epoch 1000 loss: 1.3481
Epoch 1500 loss: 1.2228
Epoch 2000 loss: 1.2297
Epoch 2500 loss: 1.2125
Epoch 3000 loss: 1.1546
Epoch 3500 loss: 1.1538
Epoch 4000 loss: 1.2010
Epoch 4500 loss: 1.1540
Epoch 5000 loss: 1.0927
Epoch 5500 loss: 1.1271
Epoch 6000 loss: 1.1377
Epoch 6500 loss: 1.1313
Epoch 7000 loss: 1.1064
Epoch 7500 loss: 1.1777
Epoch 8000 loss: 1.1365
Epoch 8500 loss: 1.1645
Epoch 9000 loss: 1.1281
Epoch 9500 loss: 1.1107


In [17]:
#evaluation: generating a new text passage
from torch.distributions.categorical import Categorical

torch.manual_seed(1)

logits = torch.tensor([[1.0, 1.0, 1.0]])

print('Probabilities:', nn.functional.softmax(logits, dim=1).numpy()[0])

m = Categorical(logits=logits)
samples = m.sample((10,))
 
print(samples.numpy())

Probabilities: [0.33333334 0.33333334 0.33333334]
[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [2]
 [1]
 [1]]


In [ ]:
torch.manual_seed(1)

logits = torch.tensor([[1.0, 1.0, 3.0]])

print('Probabilities:', nn.functional.softmax(logits, dim=1).numpy()[0])

m = Categorical(logits=logits)
samples = m.sample((10,))
 
print(samples.numpy())

In [18]:
def sample(model, starting_str, 
           len_generated_text=500, 
           scale_factor=1.0):

    encoded_input = torch.tensor([char2int[s] for s in starting_str])
    encoded_input = torch.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.eval()
    hidden, cell = model.init_hidden(1)
    hidden = hidden.to('cpu')
    cell = cell.to('cpu')
    for c in range(len(starting_str)-1):
        _, hidden, cell = model(encoded_input[:, c].view(1), hidden, cell) 
    
    last_char = encoded_input[:, -1]
    for i in range(len_generated_text):
        logits, hidden, cell = model(last_char.view(1), hidden, cell) 
        logits = torch.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        m = Categorical(logits=scaled_logits)
        last_char = m.sample()
        generated_str += str(char_array[last_char])
        
    return generated_str

torch.manual_seed(1)
model.to('cpu')
print(sample(model, starting_str='The island'))

The island containing the photoys, and in this height. He knew therefore discovered it.

“Although!”, he into which sourses between the cocks.”

some day, and though he had paped Jup had lost the cold becatively by diminous place, and how doubles, and it was evident to attempt the
reporter and his coming was surelett, just afterwards the 30th of the lake. It were hidderness, and if the
eal of its countenanches, only by a lighted lightening, and dragged
he he madely received.

“Yes, to return performed! Sp


In [19]:
logits = torch.tensor([[1.0, 1.0, 3.0]])

print('Probabilities before scaling:        ', nn.functional.softmax(logits, dim=1).numpy()[0])

print('Probabilities after scaling with 0.5:', nn.functional.softmax(0.5*logits, dim=1).numpy()[0])

print('Probabilities after scaling with 0.1:', nn.functional.softmax(0.1*logits, dim=1).numpy()[0])

Probabilities before scaling:         [0.10650698 0.10650698 0.78698605]
Probabilities after scaling with 0.5: [0.21194156 0.21194156 0.57611686]
Probabilities after scaling with 0.1: [0.3104238  0.3104238  0.37915248]


In [20]:
torch.manual_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island had been presented to the wood, which seemed to come to the water-spout of the Pacific. It was an island in the corral, and sandy must reach the last words were steamer which they had a solid in the table of the southern
specimen, they were driven to the 25th of August, the sailor had showed this was in the most impresising of a solid thirty to the sand, and the point of the island, which was struggled without any attention of some transformation.

“This was now that it would be said, the last 


In [21]:
#saving the model (state dictionary)
#saves the model's learnable parameters (weights and biases)
model_state_dict = model.state_dict()
torch.save(model_state_dict, "20240226_model.pt")

In [22]:
#saving the entire model
model.eval()
torch.save(model, "entire_model.pt")

In [ ]:
#loading the model
#torch.load()

In [23]:
#torch.manual_seed(1)
print(sample(model, starting_str='The monkeys from', scale_factor=2.0))

The monkeys from the boat appeared to be descended to the shore,
and when the sailor thought the most importance of the corral, was completely reporter.

“The ‘Nautilus,’ found them. It was a substance of
a word than the part of the subterranean fire of the stern to mark and a high continued in the corral.

In the mouth, the passengers, and as the bottom of the same time to the distance of a week the incidents of the surface of the shore, and as if
the hollowed of the sea--well, and the position of the air in t


In [24]:
print(sample(model, starting_str='The monkeys from', scale_factor=2.0))

The monkeys from the summit of the horizon was for the document was at least to far as if
we shall stand for the sudden arrived at the sandy complete and a word the colonists of that time to the sand, and the sailor and Herbert, and Neb examined the reporter, “and now that the ‘Duncan’t was not a good and descending to the works,” replied the reporter. “I will ascend the opening is an
inhabited words in the same time put to the surface of the point of the southern side
of the lake and some power of the mountain


In [25]:
print(sample(model, starting_str='The monkeys from', scale_factor=1.0))

The monkeys from Claw Capehic acid of
their specimen, and other part of the opening, and you help to escape the lake appeared to dwell
towards it up in the hours.






Hhanchieft and the settlers disputed through frightful tonds. However, the water, which beds had chose winding of, that is to say, that
doubtless, so as to begin as in widters, in a immediate time passed.

Pencroft Breaks, scrently ran was smoken.
Cyrus Harding appeared as if, to reply to put this famid river. Between the rock.
How many on the s


In [26]:
print(sample(model, starting_str='The monkeys from', scale_factor=2.5))

The monkeys from the mouth of the colonists would be finished, and as the engineer thought it was an infaction to the entrance of a sharp course at the corral was all the sailor and Neb and Herbert was there and the sailor, and Herbert, and the engineer were about to say, the colonists would have been fixed in the corral. The engineer and Neb had disappeared to be a substance to the sand, which would be the day where the plateau of the same time the one would be the balloon, in the same time to the sun. It was 


In [28]:
print(sample(model, starting_str='One day the', scale_factor=3.0))

One day the sailor was said, and the colonists had come as if the wood for the sand was still a long time to the corral. The sailor was then said to the same time to the same time as the case to the horizon of the summit of the wood. It was an
exploration of the southern coast. It was so strong to the same time as the colonists were reached the shore, and as the colonists were allowed to the sailor, “if it is so as to be a spot where the wood of the coast of the plateau of the sand with a ship in the sand,
